# Benchmarking GT4Py

Compare a vanilla Numpy and a GT4Py-powered implementation of some basic mathematical 
operations.

## User input

In [ ]:
import numpy as np

# gt4py
backend = "gtx86"
dtype = np.float64
storage_shape = (321, 321, 120)
default_origin = (3, 3, 0)

# serialization
serialize = True
filename = "timings_mac.xlsx"
sheetname = "{} x {} x {}".format(*storage_shape)
sheetidx = 0

## Data initialization

In [ ]:
# install numexpr if needed
is_installed = !pip list 2> /dev/null | grep 'numexpr'
if not is_installed:
    !pip install numexpr

In [ ]:
from copy import deepcopy
from gt4py import gtscript, storage as gt_storage
import numexpr as ne
import numpy as np
from tasmania.python.utils.gtscript_utils import (
    set_annotations,
    stencil_abs_defs,
    stencil_iabs_defs,
    stencil_copy_defs,
    stencil_copychange_defs,
    stencil_add_defs,
    stencil_iadd_defs,
    stencil_sub_defs,
    stencil_isub_defs,
    stencil_mul_defs,
    stencil_imul_defs,
    stencil_scale_defs,
    stencil_iscale_defs,
    stencil_addsub_defs,
    stencil_iaddsub_defs,
    stencil_fma_defs,
    stencil_sts_rk2_0_defs,
    stencil_sts_rk3ws_0_defs,
)
from tasmania.python.utils.storage_utils import zeros

a = zeros(storage_shape, backend, dtype, default_origin)
a[...] = np.random.rand(*storage_shape)
a_dc = deepcopy(a)
b = zeros(storage_shape, backend, dtype, default_origin)
b[...] = np.random.rand(*storage_shape)
c = zeros(storage_shape, backend, dtype, default_origin)
c[...] = np.random.rand(*storage_shape)
d = zeros(storage_shape, backend, dtype, default_origin)

f = np.random.rand(1).item()

In [ ]:
# monkey-patching numpy
gt_storage.prepare_numpy()

## Serialization setup

In [ ]:
if serialize:
    # install openpyxl if needed
    is_installed = !pip list 2> /dev/null | grep 'openpyxl'
    if not is_installed:
        !pip install openpyxl==2.6
        
    from openpyxl import Workbook, load_workbook
    from openpyxl.styles import Alignment, Font

    # create the spreadsheet if needed
    import os
    if not os.path.exists(filename):
        wb = Workbook()
        wb.save(filename=filename)
    else:
        wb = load_workbook(filename=filename)
        
    # create the sheet if needed
    if sheetname not in wb.sheetnames:
        wb.create_sheet(sheetname, sheetidx)
        
    sheet = wb[sheetname]
    
    for row in sheet["A1:H16"]:
        for cell in row:
            cell.alignment = Alignment(horizontal="center")
            cell.font = Font(name="Calibri", size="14")

    sheet.column_dimensions["A"].width = 20
    sheet["A2"]  = "copy"
    sheet["A3"]  = "copychange"
    sheet["A4"]  = "abs"
    sheet["A5"]  = "iabs"
    sheet["A6"]  = "add"
    sheet["A7"]  = "iadd"
    sheet["A8"]  = "sub"
    sheet["A9"]  = "isub"
    sheet["A10"] = "mul"
    sheet["A11"] = "imul"
    sheet["A12"] = "addsub"
    sheet["A13"] = "iaddsub"
    sheet["A14"] = "fma"
    sheet["A15"] = "sts_rk2_0"
    sheet["A16"] = "sts_rk3ws_0"

    sheet.column_dimensions["B"].width = 20
    sheet["B1"] = "numpy"
    sheet.column_dimensions["C"].width = 20
    sheet["C1"] = "numexpr"
    sheet.column_dimensions["D"].width = 20
    sheet["D1"] = "gtdebug"
    sheet.column_dimensions["E"].width = 20
    sheet["E1"] = "gtnumpy"
    sheet.column_dimensions["F"].width = 20
    sheet["F1"] = "gtx86"
    sheet.column_dimensions["G"].width = 20
    sheet["G1"] = "gtmc"
    sheet.column_dimensions["H"].width = 20
    sheet["H1"] = "gtcuda"
    
    # save
    wb.save(filename=filename)

In [ ]:
rows = {
    "copy":         2,
    "copychange":   3,
    "abs":          4,
    "iabs":         5,
    "add":          6,
    "iadd":         7,
    "sub":          8,
    "isub":         9,
    "mul":         10,
    "imul":        11,
    "addsub":      12,
    "iaddsub":     13,
    "fma":         14,
    "sts_rk2_0":   15,
    "sts_rk3ws_0": 16
}
cols = {
    "vanilla_numpy": "B",
    "numexpr":       "C",
    "debug":         "D",
    "numpy":         "E",
    "gtx86":         "F",
    "gtmc":          "G",
    "gtcuda":        "H"
}

## copy

In [ ]:
# numpy
out = %timeit -o a[...] = b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["copy"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_copy_defs, dtype)
stencil_copy = gtscript.stencil(
    backend=backend, definition=stencil_copy_defs, rebuild=False
)

out = %timeit -o stencil_copy(src=b, dst=a, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["copy"])].value = 1000. * out.average
    wb.save(filename=filename)

## copychange

In [ ]:
# numpy
out = %timeit -o a[...] = - b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["copychange"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit c[...] = ne.evaluate("-b")

In [ ]:
# numexpr
out = %timeit -o c = ne.evaluate("-b")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["copychange"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_copychange_defs, dtype)
stencil_copychange = gtscript.stencil(
    backend=backend, definition=stencil_copychange_defs, rebuild=False
)

out = %timeit -o stencil_copychange(src=b, dst=a, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["copychange"])].value = 1000. * out.average
    wb.save(filename=filename)

## abs

In [ ]:
# numpy
out = %timeit -o b[...] = np.abs(a.data)
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["abs"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_abs_defs, dtype)
stencil_abs = gtscript.stencil(
    backend=backend, definition=stencil_abs_defs, rebuild=False
)

out = %timeit -o stencil_abs(a, b, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["abs"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_abs(a, b, origin=default_origin, domain=domain)

## iabs

In [ ]:
# numpy
out = %timeit -o a[...] = np.abs(a.data)
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["iabs"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
a[...] = a_dc[...]

set_annotations(stencil_iabs_defs, dtype)
stencil_iabs = gtscript.stencil(
    backend=backend, definition=stencil_iabs_defs, rebuild=False
)

out = %timeit -o stencil_iabs(a, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["iabs"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_iabs(a, origin=default_origin, domain=domain)

## add

In [ ]:
# numpy
out = %timeit -o c[...] = a[...] + b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["add"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit c[...] = ne.evaluate("a + b")

In [ ]:
# numexpr
out = %timeit -o c = ne.evaluate("a + b")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["add"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_add_defs, dtype)
stencil_add = gtscript.stencil(
    backend=backend, definition=stencil_add_defs, rebuild=False
)

out = %timeit -o stencil_add(in_a=a, in_b=b, out_c=c, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["add"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_add(in_a=a, in_b=b, out_c=c, origin=default_origin, domain=domain)

## iadd

In [ ]:
# numpy
out = %timeit -o a[...] += b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["iadd"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
a[...] = a_dc[...]

set_annotations(stencil_iadd_defs, dtype)
stencil_iadd = gtscript.stencil(
    backend=backend, definition=stencil_iadd_defs, rebuild=False
)

out = %timeit -o stencil_iadd(inout_a=a, in_b=b, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["iadd"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_iadd(inout_a=a, in_b=b, origin=default_origin, domain=domain)

## sub

In [ ]:
# numpy
out = %timeit -o c[...] = a[...] - b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["sub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit c[...] = ne.evaluate("a - b")

In [ ]:
# numexpr
out = %timeit -o c = ne.evaluate("a - b")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["sub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_sub_defs, dtype)
stencil_sub = gtscript.stencil(
    backend=backend, definition=stencil_sub_defs, rebuild=False
)

out = %timeit -o stencil_sub(in_a=a, in_b=b, out_c=c, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["sub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_sub(in_a=a, in_b=b, out_c=c, origin=default_origin, domain=domain)

## isub

In [ ]:
# numpy
out = %timeit -o a[...] -= b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["isub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
a[...] = a_dc[...]

set_annotations(stencil_isub_defs, dtype)
stencil_isub = gtscript.stencil(
    backend=backend, definition=stencil_isub_defs, rebuild=False
)

out = %timeit -o stencil_isub(inout_a=a, in_b=b, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["isub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_isub(inout_a=a, in_b=b, origin=default_origin, domain=domain)

## mul

In [ ]:
# numpy
out = %timeit -o c[...] = a[...] * b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["mul"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit c[...] = ne.evaluate("a * b")

In [ ]:
# numexpr
out = %timeit -o c = ne.evaluate("a * b")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["mul"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_mul_defs, dtype)
stencil_mul = gtscript.stencil(
    backend=backend, definition=stencil_mul_defs, rebuild=False
)

out = %timeit -o stencil_mul(in_a=a, in_b=b, out_c=c, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["mul"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_mul(in_a=a, in_b=b, out_c=c, origin=default_origin, domain=domain)

## imul

In [ ]:
# numpy
out = %timeit -o a[...] *= b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["imul"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
a[...] = a_dc[...]

set_annotations(stencil_imul_defs, dtype)
stencil_imul = gtscript.stencil(
    backend=backend, definition=stencil_imul_defs, rebuild=False
)

out = %timeit -o stencil_imul(inout_a=a, in_b=b, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["imul"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_imul(inout_a=a, in_b=b, origin=default_origin, domain=domain)

## addsub

In [ ]:
# numpy
out = %timeit -o d[...] = a[...] + b[...] - c[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["addsub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit d[...] = ne.evaluate("a + b - c")

In [ ]:
# numexpr
out = %timeit -o d = ne.evaluate("a + b - c")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["addsub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_addsub_defs, dtype)
stencil_addsub = gtscript.stencil(
    backend=backend, definition=stencil_addsub_defs, rebuild=False
)

out = %timeit -o stencil_addsub(in_a=a, in_b=b, in_c=c, out_d=d, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["addsub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_addsub(in_a=a, in_b=b, in_c=c, out_d=d, origin=default_origin, domain=domain)

## iaddsub

In [ ]:
# numpy
out = %timeit -o a[...] += b[...] - c[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["iaddsub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
a[...] = a_dc[...]

stencil_iaddsub = gtscript.stencil(
    backend=backend, definition=stencil_iaddsub_defs, rebuild=False
)

out = %timeit -o stencil_iaddsub(inout_a=a, in_b=b, in_c=c, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["iaddsub"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
a[...] = a_dc[...]
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_iaddsub(inout_a=a, in_b=b, in_c=c, origin=default_origin, domain=domain)

## fma

In [ ]:
# numpy
out = %timeit -o c[...] = a[...] + f * b[...]
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["fma"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit c[...] = ne.evaluate("a + f * b")

In [ ]:
# numexpr
out = %timeit -o c = ne.evaluate("a + f * b")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["fma"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_fma_defs, dtype)
stencil_fma = gtscript.stencil(
    backend=backend, definition=stencil_fma_defs, rebuild=False
)

out = %timeit -o stencil_fma(in_a=a, in_b=b, out_c=c, f=f, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["fma"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_fma(in_a=a, in_b=b, out_c=c, f=f, origin=default_origin, domain=domain)

## sts_rk2_0

In [ ]:
# numpy
out = %timeit -o d[...] = 0.5 * (a[...] + b[...] + f * c[...])
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["sts_rk2_0"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit d[...] = ne.evaluate("0.5 * (a + b + f * c)")

In [ ]:
# numexpr
out = %timeit -o d = ne.evaluate("0.5 * (a + b + f * c)")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["sts_rk2_0"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_sts_rk2_0_defs, dtype)
stencil_sts_rk2_0 = gtscript.stencil(
    backend=backend, definition=stencil_sts_rk2_0_defs, rebuild=False
)

out = %timeit -o stencil_sts_rk2_0(a, b, c, d, dt=f, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["sts_rk2_0"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_sts_rk2_0(a, b, c, d, dt=f, origin=default_origin, domain=domain)

## sts_rk3ws_0

In [ ]:
# numpy
out = %timeit -o d[...] = (2.0 * a[...] + b[...] + f * c[...]) / 3.0
out

if serialize:
    sheet["{}{}".format(cols["vanilla_numpy"], rows["sts_rk3ws_0"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# numexpr with assignment
%timeit d[...] = ne.evaluate("(2.0 * a + b + f * c) / 3.0")

In [ ]:
# numexpr
out = %timeit -o d = ne.evaluate("(2.0 * a + b + f * c) / 3.0")
out

if serialize:
    sheet["{}{}".format(cols["numexpr"], rows["sts_rk3ws_0"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py
set_annotations(stencil_sts_rk3ws_0_defs, dtype)
stencil_sts_rk3ws_0 = gtscript.stencil(
    backend=backend, definition=stencil_sts_rk3ws_0_defs, rebuild=False
)

out = %timeit -o stencil_sts_rk3ws_0(a, b, c, d, dt=f, origin=(0, 0, 0), domain=storage_shape)
out

if serialize:
    sheet["{}{}".format(cols[backend], rows["sts_rk3ws_0"])].value = 1000. * out.average
    wb.save(filename=filename)

In [ ]:
# gt4py: make origin coincide with default_origin
domain = tuple(s - 2*d for s, d in zip(storage_shape, default_origin))
%timeit stencil_sts_rk3ws_0(a, b, c, d, dt=f, origin=default_origin, domain=domain)
